# How Can One Save an Instance of an Estimator?

I have a situation in which I want to use an estimator that has been fit in an original Notebook to predict new values given data in a new Notebook.  The question is, can I save a specific model instance to be read in elsewhere?  This Notebook will run a basic regression on simulated data.  We will then attempt to write the model to disk, read back in, and use it to predict outcomes on new data.

In [1]:
#Basic data manipulation
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

#Estimating methods
import statsmodels.formula.api as smf
from statsmodels.api import add_constant

#Serialization
import pickle

## Simulation of Input Data

We are going to follow a very basic model:

$Y = X_1 + 2X_2 + \epsilon$

In [2]:
#Generate random vectors for regressors
x1,x2=(np.random.uniform(20,size=50),np.random.uniform(20,size=50))

#Generate dependent variable
y=x1 + 2*x2 + [np.random.uniform(3) for val in range(len(x1))]

#Capture data in DF
data=DataFrame({'y':y,
                'x1':x1,
                'x2':x2})

data

,x1,x2,y
0,6.426795,18.526146,46.194408
1,9.970794,4.286937,21.371893
2,13.526982,16.869215,49.199389
3,14.124276,3.378121,22.956715
4,12.372136,4.677349,23.956064
5,11.513103,18.378873,50.134025
6,12.725635,17.346911,50.298347
7,5.826384,19.403542,46.016669
8,15.655518,4.671959,27.916892
9,12.905911,4.872422,23.686621


## Estimation of the Model

We are just going to fit a standard OLS estimator...

In [3]:
#Fit model
model=smf.ols(formula='y ~ x1 + x2', data=data).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.232e+04
Date:                Fri, 26 Feb 2016   Prob (F-statistic):           1.17e-64
Time:                        20:15:31   Log-Likelihood:                -44.204
No. Observations:                  50   AIC:                             94.41
Df Residuals:                      47   BIC:                             100.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      2.2185      0.252      8.789      0.000         1.711     2.726
x1             0.9950      0.017     58.379      0.000         0.961     1.029
x2             1.9848      0.015    134.216      0.000         1.955     2.015
==============================================================================
Omnibus:                        9.250   Durbin-Watson:                   2.216
Prob(Omnibus):                  0.010   Jarque-Bera (JB):                2.707
Skew:                          -0.024   Prob(JB):                        0.258
Kurtosis:                       1.861   Cond. No.                         51.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

To provide a point of comparison, we will use the regressor subset to predict `y`.

In [4]:
pred_y=model.predict(data[['x1','x2']])

pred_y

array([ 45.3830118 ,  20.64783514,  49.15899802,  22.97671632,
        23.81201359,  50.15152271,  49.30977897,  46.52703118,
        27.06824106,  24.73028715,  55.73780948,  46.14598452,
        35.04579904,  42.77663411,  24.92214325,  34.17396757,
        21.78158769,  10.54739915,  38.82710887,  30.57087857,
        38.03462221,  40.5112197 ,  44.88486214,  46.76047541,
        48.71681776,  34.14337941,  59.28093668,  47.6069733 ,
        46.00559424,  47.98132577,  37.90945678,  14.59101547,
        26.23134675,  54.9390063 ,  36.14020639,  39.90891943,
        29.2219375 ,  17.5978753 ,   9.94806419,  28.35825447,
        32.16683832,  46.99007463,  46.67503341,  45.96060996,
        34.28821209,  53.22079473,  33.37406911,  11.29626885,
        57.70762237,   6.95323106])

## Writing the Model to Disk

Now that we have instantiated this model object, let's see if we can write it to disk.  To do this, we are going to use a method called "pickling", which serializes an object into a byte stream so that it may be written to disk.  In theory, we can read this same byte stream back elsewhere, and use it to reconstitute the object that had been written to disk.

So, what exactly are we writing to disk?  Here is the ASCII version of the byte stream:

In [5]:
pickle.dumps(model)

'ccopy_reg\n_reconstructor\np0\n(cstatsmodels.regression.linear_model\nRegressionResultsWrapper\np1\nc__builtin__\nobject\np2\nNtp3\nRp4\n(dp5\nS\'__doc__\'\np6\nS\'\\n    Results class for for an OLS model.\\n\\n    Most of the methods and attributes are inherited from RegressionResults.\\n    The special methods that are only available for OLS are:\\n\\n    - get_influence\\n    - outlier_test\\n    - el_test\\n    - conf_int_el\\n\\n    See Also\\n    --------\\n    RegressionResults\\n\\n    \'\np7\nsS\'_results\'\np8\ng0\n(cstatsmodels.regression.linear_model\nOLSResults\np9\ng2\nNtp10\nRp11\n(dp12\nS\'normalized_cov_params\'\np13\ncnumpy.core.multiarray\n_reconstruct\np14\n(cnumpy\nndarray\np15\n(I0\ntp16\nS\'b\'\np17\ntp18\nRp19\n(I1\n(L3L\nL3L\ntp20\ncnumpy\ndtype\np21\n(S\'f8\'\np22\nI0\nI1\ntp23\nRp24\n(I3\nS\'<\'\np25\nNNNI-1\nI-1\nI0\ntp26\nbI00\nS\'\\xd4\\xb4\\xb9\\xb5\\xa5W\\xc6?8\\xddi\\x84\\xc66\\x80\\xbf\\xd9\\xa1\\x9a\\xad+\\xa1v\\xbf8\\xddi\\x84\\xc66\\x80\\xbf\\xf4\

I guess I am just trusting that this is correct.  Let's write that nonsense to disk.

In [6]:
with open('model_dump','w') as f:
    pickle.dump(model,f)

Now we can read the model object back in, save it as a new instance, and use it to predict based upon the original regressor data.  This result can be compared to the original prediction above.

In [7]:
#Capture model object from disk
with open('model_dump','r') as f:
    model2=pickle.load(f)

#Show model summary
print model2.summary()

#Predict new values of y
# pred_y2=model2.predict(data[['x1','x2']])

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.232e+04
Date:                Fri, 26 Feb 2016   Prob (F-statistic):           1.17e-64
Time:                        20:15:31   Log-Likelihood:                -44.204
No. Observations:                  50   AIC:                             94.41
Df Residuals:                      47   BIC:                             100.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      2.2185      0.252      8.789      0.0

    ---------------------------------------------------------------------------
    AttributeError                            Traceback (most recent call last)
    <ipython-input-16-f7a79b10e2b5> in <module>()
          7 
          8 #Predict new values of y
    ----> 9 pred_y2=model2.predict(data[['x1','x2']])

    C:\Users\marvinw\AppData\Local\Continuum\Anaconda\lib\site-packages\statsmodels\base\model.pyc in predict(self, exog, transform, *args, **kwargs)
        876         if transform and hasattr(self.model, 'formula') and exog is not None:
        877             from patsy import dmatrix
    --> 878             exog = dmatrix(self.model.data.orig_exog.design_info.builder,
        879                     exog)
        880         return self.model.predict(self.params, exog, *args, **kwargs)

    C:\Users\marvinw\AppData\Local\Continuum\Anaconda\lib\site-packages\pandas\core\generic.pyc in __getattr__(self, name)
       1976                 return self[name]
       1977             raise AttributeError("'%s' object has no attribute '%s'" %
    -> 1978                                  (type(self).__name__, name))
       1979 
       1980     def __setattr__(self, name, value):

    AttributeError: 'DataFrame' object has no attribute 'design_info'

Hmmmm.  While it does appear that we have reconstituted the object, clearly something was lost in translation.  This helpful [SO post](http://stackoverflow.com/questions/20724919/pandas-dataframe-attributeerror-dataframe-object-has-no-attribute-design-inf) reveals that user additions to the model can get lost.  In this case, `design_info` is referring to the formula used to fit the model: `'y ~ x1 + x2'`.  The post suggests either building a design matrix explicitly with **`patsy`**, or just making sure we add a constant to our input data for the unpickled version of the model.

In [8]:
#Predict new values of y
pred_y2=model2.predict(add_constant(data[['x1','x2']]),transform=False)

pred_y-pred_y2

array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   3.55271368e-15,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   7.10542736e-15,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,  -7.10542736e-15,   0.00000000e+00,
         3.55271368e-15,   3.55271368e-15,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,  -7.10542736e-15,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -1.77635684e-15,
         0.00000000e+00,

I think that I can handle those small deviations.  Looks like this works as expected.

In [14]:
np.random.choice(range(0,2),size=10)

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0])

In [15]:
import statsmodels.api as sm
X=data[['x1','x2']].values
y=np.random.choice(range(0,2),size=len(data))
p=sm.Probit(y,X).fit(maxiter=1000)
p

Optimization terminated successfully.
         Current function value: 0.687744
         Iterations 4


In [62]:
summary_string=p.summary().as_csv().split(',')
ll_text_idx=[i for i,content in enumerate(summary_string) if 'Log-Likelihood:' in content][0]
ll_rep_value=summary_string[conv_text_idx+1].strip().split('\n')[0]
ll_max_found=ll_rep_value != 'nan'
ll_max_found

True

In [40]:
p.summary().as_csv().split(',')

['                  Probit Regression Results                   \nDep. Variable:',
 'y               ',
 '  No. Observations:  ',
 '    50  \nModel:        ',
 'Probit          ',
 '  Df Residuals:      ',
 '    48  \nMethod:       ',
 'MLE             ',
 '  Df Model:          ',
 '     1  \nDate:         ',
 'Mon',
 ' 29 Feb 2016',
 '  Pseudo R-squ.:     ',
 '-0.02189\nTime:         ',
 '16:09:48        ',
 '  Log-Likelihood:    ',
 ' -34.387\nconverged:    ',
 'True            ',
 '  LL-Null:           ',
 ' -33.651\n              ',
 '                ',
 '  LLR p-value:       ',
 ' 1.000  \n  ',
 '   coef   ',
 ' std err ',
 '    z    ',
 'P>|z| ',
 ' [95.0% Conf. Int.]\nx1',
 '   -0.0166',
 '    0.027',
 '   -0.625',
 ' 0.532',
 '   -0.069     0.035\nx2',
 '    0.0085',
 '    0.026',
 '    0.327',
 ' 0.743',
 '   -0.042     0.059']

In [42]:
print p.summary().tables[0]

                          Probit Regression Results                           
Dep. Variable:                      y   No. Observations:                   50
Model:                         Probit   Df Residuals:                       48
Method:                           MLE   Df Model:                            1
Date:                Mon, 29 Feb 2016   Pseudo R-squ.:                -0.02189
Time:                        16:11:05   Log-Likelihood:                -34.387
converged:                       True   LL-Null:                       -33.651
                                        LLR p-value:                     1.000


In [31]:
dir(p.summary().tables[0])

['_Cell',
 '_Row',
 '__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__delslice__',
 '__dict__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getslice__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__setslice__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_headers_stubs',
 '_colwidths',
 '_csv_fmt',
 '_data2rows',
 '_datatypes',
 '_get_colwidths',
 '_get_fmt',
 '_html_fmt',
 '_latex_fmt',
 '_repr_html_',
 '_txt_fmt',
 'append',
 'as_csv',
 'as_html',
 'as_latex_tabular',
 'as_text',
 'count',
 'data',
 'extend',
 'extend_right',
 'get_colwidths',
 'index',
 'insert',
 'insert_header_row',
 'insert_stubs',
 'label_cells',
 'output_formats',
 'pad',
 'pop',
 'remove',